In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import pyomo.environ as pyo
import numpy as np
from pyomo.opt import SolverFactory
from parser import parse_tsplib
from plotter import plot_tsp_solution, print_route_summary

opt = { # Please dont steal my credentials
    "WLSACCESSID": '60760e28-aa6e-4c6c-af3f-469592dc6bec',
    "WLSSECRET": '95fff45d-3d51-4958-b65c-7bbc487d2e09',
    "LICENSEID": 2707350,
}

# Problem 1

## Theory

The Traveling Salesman Problem (TSP), often expressed using the Miller–Tucker–Zemlin (MTZ) formulation, is a classic problem in optimization and graph theory. It aims to find the shortest possible route that visits each destination node exactly once and returns to the starting point (origin). In graph theory, this corresponds to a minimum-cost Hamiltonian cycle, which is a closed path that visits every vertex once. Such a cycle always exists if the network is complete, meaning every pair of nodes is linked by an edge.

The MTZ part is there to handle the elimination of subtours, it does this by introducing a new variable $u_i$ for each node we need to visit (excluding the origin) and this variable will corespond with the order we will visit the nodes. It is expressed as following:

$u_i - u_j + px_{i,j} \leq p-1 \quad (1 \leq i \neq j \leq n)$

This will thus eliminate all subtours because for a subtour to exist there need to exist a cycle from $x_{a,b},\ldots,x_{g,a}$ where all the $x$'s are 1 but this contradicts the increasing nature of the constraint. One could say that the MTZ forces the model to find a path between all the destination nodes. Notice that $p\cdot t \geq n$ this is to ensure the routes are long enough

The TSP can thus be formulated as following when $t=1$ else its a Vehicle Routing Problem (VRP)
$$
\min_{x} \sum_{i=0}^{n}\sum_{j=0,\\ i\neq j}^{n} c_{i,j} x_{i,j} \\
s.t \\
\sum_{i=0, i\neq j}^{n} x_{i,j} = 1 \quad (j = 1,\ldots,n) \quad \text{Inflow constraint} \\
\sum_{j=0, j\neq i}^{n} x_{i,j} = 1 \quad (i = 1,\ldots,n) \quad \text{Outflow constraint} \\
u_i - u_j + px_{i,j} \leq p-1 \quad (1 \leq i \neq j \leq n) \quad \text{MTZ subtour elimination}  \\
\sum_{i=1}^{n} x_{i,0} = t \quad \text{Inflow constraint from origin} \\
\sum_{j=1}^{n} x_{0,j} = t \quad \text{Outflow constraint from origin} \\
x_{i,j} \in \{0,1\} \quad \forall i,j\\
u_{j} \in Z^{+} \quad \forall j

$$
Observe I use 0 indexing to handle the origin, Assume aswell that there are no self loop and that we are working with the standard traveling salesman problem where we only allow one tour from the origin, $t=1$ and can thus let $p=n$.
$$
\min_{x} \sum_{i=0}^{n}\sum_{j=0,\\ i\neq j}^{n} c_{i,j} x_{i,j} \\
s.t \\
\sum_{i=0, i\neq j}^{n} x_{i,j} = 1 \quad (j = 0,\ldots,n) \quad \text{Inflow constraint} \\
\sum_{j=0, j\neq i}^{n} x_{i,j} = 1 \quad (i = 0,\ldots,n) \quad \text{Outflow constraint} \\
u_i - u_j + nx_{i,j} \leq n-1 \quad (1 \leq i \neq j \leq n) \quad \text{MTZ subtour elimination}\\
x_{i,j} \in \{0,1\} \quad \forall i,j\\
u_{j} \in Z^{+} \quad \forall j
$$

## Code

In [ ]:
def build_tsp_mtz(dist) -> pyo.Model: # GPT-5 GENERATED with alot of modifications
    """
    Pyomo MTZ TSP model (depot = 0), with clean domains (no 'if' inside rules).
    Args:
        dist: square cost/distance matrix-like (n x n), index 0..n-1
    Returns:
        pyo.Model
    """
    n = len(dist)
    P = n  # MTZ constant

    m = pyo.ConcreteModel()

    # --- Sets (domains) ---
    m.V   = pyo.Set(initialize=range(n), ordered=True)                # all nodes {0..n-1}
    m.Vnd = pyo.Set(initialize=range(1, n), ordered=True)             # non-depot nodes {1..n-1}
    m.A   = pyo.Set(dimen=2, initialize=[(i, j) for i in range(n) for j in range(n) if i != j])
    # pairs used in MTZ (exclude depot and diagonals)
    m.MTZPairs = pyo.Set(dimen=2, initialize=[(i, j) for i in range(1, n) for j in range(1, n) if i != j])

    # --- Parameters ---
    def c_rule(m, i, j):
        return float(dist[i][j])
    m.c = pyo.Param(m.A, initialize=c_rule, within=pyo.NonNegativeReals, mutable=False)

    # --- Decision variables ---
    m.x = pyo.Var(m.A, within=pyo.Binary)   # arc selection
    m.u = pyo.Var(m.V, within=pyo.Reals)    # MTZ order variables

    # --- Objective ---
    m.obj = pyo.Objective(expr=sum(m.c[i, j] * m.x[i, j] for (i, j) in m.A), sense=pyo.minimize)

    # --- Degree constraints (enter once, leave once) ---
    m.outdeg = pyo.Constraint(m.V, rule=lambda m, i: sum(m.x[i, j] for j in m.V if j != i) == 1)
    m.indeg  = pyo.Constraint(m.V, rule=lambda m, j: sum(m.x[i, j] for i in m.V if i != j) == 1)

    # --- MTZ: fix depot and bound non-depot u's WITHOUT 'if' in rules --- # UNECESSARY GPT GENERATED 
    # Fix depot order
    #m.u[0].fix(1)
    # Bounds for non-depot nodes via constraints (instead of conditional bounds rule)
    #m.u_lb = pyo.Constraint(m.Vnd, rule=lambda m, i: m.u[i] >= 2)
    #m.u_ub = pyo.Constraint(m.Vnd, rule=lambda m, i: m.u[i] <= P)

    # MTZ subtour-elimination (domain already excludes depot and i==j)
    m.mtz = pyo.Constraint(m.MTZPairs, rule=lambda m, i, j: m.u[i] - m.u[j] + P * m.x[i, j] <= P - 1)

    # Optional Gurobi suffixes (import duals/reduced costs if needed)
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    m.rc   = pyo.Suffix(direction=pyo.Suffix.IMPORT)

    return m


In [ ]:
for problem in ['berlin52.tsp', 'lin318.tsp', 'st70.tsp', 'ts225', 'ulysses16.tsp', 'ulysses22.tsp']:
    data = parse_tsplib(f'Core/{problem}') # Data parsing
    
    model = build_tsp_mtz(data['dist']) # Model building

    with SolverFactory( # Model Solving
    'gurobi', solver_io='python', manage_env=True, options=opt
    ) as solver:
        solver.solve(
            model,
            tee=True,
            options={
                #"Method": 1,        # Dual Simplex (avoid barrier)
                #"Crossover": 0,     # No crossover
                #"Presolve": 2,
                #"MIPFocus": 1,      # Focus on finding feasible routes
                #"Heuristics": 0.2,
                #"Cuts": 2,
                #"Symmetry": 2,
                "TimeLimit": 300,
                #"MIPGap": 0.01,
                # "Threads": 0,     # use all cores (uncomment if you want to pin threads)
                # "LogFile": "gurobi_cvrp.log",
            }
        ) 

    plot_tsp_solution(model, data['coords'], title=f'TSP MTZ Solution for {problem}') # Plotting
    print_route_summary(model, data['dist']) # Route summary

KeyError: 'dist_matrix'